In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from animalclef.spark import get_spark
from pyspark.sql import functions as F

spark = get_spark(cores=4, memory="10g")
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 04:14:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/02 04:14:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
25/03/02 04:14:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
! ls ~/scratch/animalclef

animal-clef-2025.zip  images  metadata.csv  sample_submission.csv


In [5]:
from pathlib import Path

root = Path("~/scratch/animalclef").expanduser()
metadata = spark.read.csv(f"{root}/metadata.csv", header=True)
metadata.printSchema()
metadata.show()

root
 |-- image_id: string (nullable = true)
 |-- identity: string (nullable = true)
 |-- path: string (nullable = true)
 |-- date: string (nullable = true)
 |-- orientation: string (nullable = true)
 |-- species: string (nullable = true)
 |-- split: string (nullable = true)
 |-- dataset: string (nullable = true)

+--------+------------------+--------------------+----+-----------+-------+--------+----------+
|image_id|          identity|                path|date|orientation|species|   split|   dataset|
+--------+------------------+--------------------+----+-----------+-------+--------+----------+
|       0|LynxID2025_lynx_37|images/LynxID2025...|NULL|      right|   lynx|database|LynxID2025|
|       1|LynxID2025_lynx_37|images/LynxID2025...|NULL|       left|   lynx|database|LynxID2025|
|       2|LynxID2025_lynx_49|images/LynxID2025...|NULL|       left|   lynx|database|LynxID2025|
|       3|              NULL|images/LynxID2025...|NULL|       back|   lynx|   query|LynxID2025|
|       4|Ly

In [6]:
metadata.show(5, truncate=False)

+--------+------------------+-----------------------------------------------------------------------------------------------+----+-----------+-------+--------+----------+
|image_id|identity          |path                                                                                           |date|orientation|species|split   |dataset   |
+--------+------------------+-----------------------------------------------------------------------------------------------+----+-----------+-------+--------+----------+
|0       |LynxID2025_lynx_37|images/LynxID2025/database/000f9ee1aad063a4485379ec06741251344f57ba0e8e6546dfc4c3b83f28c0db.jpg|NULL|right      |lynx   |database|LynxID2025|
|1       |LynxID2025_lynx_37|images/LynxID2025/database/0020edb6689e9f78462394d5a6447112e66ac603f30abaacbd0e9787361acc10.jpg|NULL|left       |lynx   |database|LynxID2025|
|2       |LynxID2025_lynx_49|images/LynxID2025/database/003152e4145b5b6940091d5c1290d023ed3ad81400482b97bd2ea4ba35dbbd6b.jpg|NULL|left       |lyn

In [7]:
@F.udf("string")
def ext(path):
    return Path(path).suffix


metadata.groupBy(ext("path")).count().show(5, truncate=False)

+---------+-----+
|ext(path)|count|
+---------+-----+
|.jpg     |7271 |
|.jpeg    |1489 |
|.JPG     |6449 |
+---------+-----+



In [8]:
# images

images = (
    spark.read.format("binaryFile")
    # image/{dataset}/{split}/{image}
    .option("pathGlobFilter", "*")
    .option("recursiveFileLookup", True)
    .load(f"{root}/images")
)
images.printSchema()
images.count()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



15209

In [ ]:
@F.udf("string")
def relative_path(path, root=root):
    return Path(path.split("file:")[1]).relative_to(Path(root).absolute()).as_posix()


images.drop("content").select(relative_path("path")).show(5, truncate=False)

+-----------------------------------------------------------------------------------------------+
|relative_path(path)                                                                            |
+-----------------------------------------------------------------------------------------------+
|images/LynxID2025/database/8742db46ed00642317b899925e22d161e233f353aa87cb8ea30eaa9c3e618c90.jpg|
|images/LynxID2025/database/ade229875db3fcfbae1731aa3fd7bed8a669c02cb6e021915bc36880f7a7c47d.jpg|
|images/LynxID2025/database/bb432782a3ee2738b3681d066f459b95287f2f89a0128682f2b57dad5045cc06.jpg|
|images/LynxID2025/database/72c098e07acc5a3705a914b1ed36366e47e22b24262c7e3fe3041d8125951135.jpg|
|images/LynxID2025/database/6835b17a615deb93f61b072bdf98c7e8a714e52c60758d467e4bbc28699fc232.jpg|
+-----------------------------------------------------------------------------------------------+
only showing top 5 rows

